In [1]:
import os
import os.path as osp
import shutil
import glob

import torch
from torch_geometric.data import InMemoryDataset, download_url, extract_zip
from torch_geometric.io import read_off

def download(root='data/modelnet/', name='40'):
    urls = {
        '10':
        'http://vision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip',
        '40': 'http://modelnet.cs.princeton.edu/ModelNet40.zip'
    }
    os.makedirs(root, exist_ok=True)
    path = download_url(urls[name], root)
    extract_zip(path, root)
    os.unlink(path)
    folder = osp.join(root, 'ModelNet{}'.format(self.name))
    shutil.rmtree(self.raw_dir)
    os.rename(folder, self.raw_dir)

    # Delete osx metadata generated during compression of ModelNet10
    metadata_folder = osp.join(root, '__MACOSX')
    if osp.exists(metadata_folder):
        shutil.rmtree(metadata_folder)

In [ ]:
#download()

Extracting data/modelnet/ModelNet40.zip


In [28]:
import numpy as np
import h5py
from tqdm.notebook import tqdm
import os
def extract(path):
    f = open(path, 'r')
    lines = f.readlines()
    if lines[0] == 'OFF\n':
        num = int(float(lines[1].split(" ")[0]))
        pts = []
        for i in range(2, 2+num):
            temp = lines[i][:-1].split(' ')
            pts.append([float(temp[0]), float(temp[1]), float(temp[2])])
    else:
        num = int(float(lines[0].split(" ")[0][3:]))
        pts = []
        for i in range(1, 1+num):
            temp = lines[i][:-1].split(' ')
            pts.append([float(temp[0]), float(temp[1]), float(temp[2])])
    return pts

def create_pt(base = 'data/modelnet/ModelNet40/', output_base = 'data/modelnet/modelnet_points/'):
    paths = {}
    for obj in os.listdir(base):
        if obj != '.DS_Store' and obj != 'README.txt':
            obj_base_train = base + obj + '/train/'
            obj_base_test = base + obj + '/test/'
            paths[obj] = []
            for sample in os.listdir(obj_base_train):
                paths[obj].append(obj_base_train + sample)
            for sample in os.listdir(obj_base_test):
                paths[obj].append(obj_base_test + sample)
                
    os.makedirs(output_base, exist_ok=True)

    for obj in tqdm(paths.keys()):
        h5f = h5py.File(output_base + str(obj)+'.h5', 'w')
        for path, i in zip(paths[obj], range(len(paths[obj]))):
            if path[-3:] == 'off':
                temp = np.array(extract(path))
                h5f.create_dataset('object_' + str(i), data=temp)
        h5f.close()

In [30]:
create_pt()

  0%|          | 0/40 [00:00<?, ?it/s]

In [31]:
from src import graph_construct
import os
import numpy as np
import h5py
from tqdm.notebook import tqdm

def construct_graph_knn(pts_num = 1000, base = 'data/modelnet/modelnet_points/', k=15):
    output_base = 'data/modelnet/modelnet_graph_k{x}'.format(x=k)
    os.makedirs(output_base, exist_ok=True)
    for obj in tqdm(os.listdir(base)):
        cat = obj[:-3]
        print(obj)
        if obj[-2:] == 'h5':
            os.makedirs(output_base + '/' + cat, exist_ok=True)
            f = h5py.File(base + obj, 'r')
            for key in f.keys():
                if f[key][:].shape[0] >= pts_num:
                    pts = graph_construct.pts_norm(graph_construct.pts_sample(f[key][:], pts_num))
                    if np.isnan(pts).any():
                        continue
                    temp = graph_construct.graph_construct_kneigh(pts, k=k)
                    filename = output_base + '/' + cat + '/' + key + '.h5'
                    print(filename)
                    out = h5py.File(filename, 'w')
                    out.create_dataset('edges', data=temp[0])
                    out.create_dataset('edge_weight', data=temp[1])
                    out.create_dataset('nodes', data=pts)
                    out.close()
    print('After processing, numbers of samples for each category are:')
    for obj in os.listdir(output_base):
        print(obj, len(os.listdir(output_base + obj)))

In [32]:
construct_graph_knn()

  0%|          | 0/40 [00:00<?, ?it/s]

xbox.h5
data/modelnet/modelnet_graph_k15/xbox/object_0.h5
data/modelnet/modelnet_graph_k15/xbox/object_1.h5
data/modelnet/modelnet_graph_k15/xbox/object_10.h5
data/modelnet/modelnet_graph_k15/xbox/object_100.h5
data/modelnet/modelnet_graph_k15/xbox/object_101.h5
data/modelnet/modelnet_graph_k15/xbox/object_103.h5
data/modelnet/modelnet_graph_k15/xbox/object_104.h5
data/modelnet/modelnet_graph_k15/xbox/object_105.h5
data/modelnet/modelnet_graph_k15/xbox/object_106.h5
data/modelnet/modelnet_graph_k15/xbox/object_107.h5
data/modelnet/modelnet_graph_k15/xbox/object_109.h5
data/modelnet/modelnet_graph_k15/xbox/object_11.h5
data/modelnet/modelnet_graph_k15/xbox/object_110.h5
data/modelnet/modelnet_graph_k15/xbox/object_111.h5
data/modelnet/modelnet_graph_k15/xbox/object_113.h5
data/modelnet/modelnet_graph_k15/xbox/object_114.h5
data/modelnet/modelnet_graph_k15/xbox/object_115.h5
data/modelnet/modelnet_graph_k15/xbox/object_116.h5
data/modelnet/modelnet_graph_k15/xbox/object_117.h5
data/model

data/modelnet/modelnet_graph_k15/chair/object_154.h5
data/modelnet/modelnet_graph_k15/chair/object_156.h5
data/modelnet/modelnet_graph_k15/chair/object_157.h5
data/modelnet/modelnet_graph_k15/chair/object_158.h5
data/modelnet/modelnet_graph_k15/chair/object_159.h5
data/modelnet/modelnet_graph_k15/chair/object_160.h5
data/modelnet/modelnet_graph_k15/chair/object_161.h5
data/modelnet/modelnet_graph_k15/chair/object_162.h5
data/modelnet/modelnet_graph_k15/chair/object_163.h5
data/modelnet/modelnet_graph_k15/chair/object_164.h5
data/modelnet/modelnet_graph_k15/chair/object_165.h5
data/modelnet/modelnet_graph_k15/chair/object_166.h5
data/modelnet/modelnet_graph_k15/chair/object_167.h5
data/modelnet/modelnet_graph_k15/chair/object_168.h5
data/modelnet/modelnet_graph_k15/chair/object_169.h5
data/modelnet/modelnet_graph_k15/chair/object_17.h5
data/modelnet/modelnet_graph_k15/chair/object_170.h5
data/modelnet/modelnet_graph_k15/chair/object_171.h5
data/modelnet/modelnet_graph_k15/chair/object_1

data/modelnet/modelnet_graph_k15/chair/object_320.h5
data/modelnet/modelnet_graph_k15/chair/object_321.h5
data/modelnet/modelnet_graph_k15/chair/object_322.h5
data/modelnet/modelnet_graph_k15/chair/object_323.h5
data/modelnet/modelnet_graph_k15/chair/object_324.h5
data/modelnet/modelnet_graph_k15/chair/object_325.h5
data/modelnet/modelnet_graph_k15/chair/object_326.h5
data/modelnet/modelnet_graph_k15/chair/object_327.h5
data/modelnet/modelnet_graph_k15/chair/object_328.h5
data/modelnet/modelnet_graph_k15/chair/object_329.h5
data/modelnet/modelnet_graph_k15/chair/object_33.h5
data/modelnet/modelnet_graph_k15/chair/object_330.h5
data/modelnet/modelnet_graph_k15/chair/object_331.h5
data/modelnet/modelnet_graph_k15/chair/object_332.h5
data/modelnet/modelnet_graph_k15/chair/object_333.h5
data/modelnet/modelnet_graph_k15/chair/object_334.h5
data/modelnet/modelnet_graph_k15/chair/object_336.h5
data/modelnet/modelnet_graph_k15/chair/object_337.h5
data/modelnet/modelnet_graph_k15/chair/object_3

data/modelnet/modelnet_graph_k15/chair/object_478.h5
data/modelnet/modelnet_graph_k15/chair/object_479.h5
data/modelnet/modelnet_graph_k15/chair/object_48.h5
data/modelnet/modelnet_graph_k15/chair/object_481.h5
data/modelnet/modelnet_graph_k15/chair/object_482.h5
data/modelnet/modelnet_graph_k15/chair/object_483.h5
data/modelnet/modelnet_graph_k15/chair/object_484.h5
data/modelnet/modelnet_graph_k15/chair/object_485.h5
data/modelnet/modelnet_graph_k15/chair/object_486.h5
data/modelnet/modelnet_graph_k15/chair/object_488.h5
data/modelnet/modelnet_graph_k15/chair/object_489.h5
data/modelnet/modelnet_graph_k15/chair/object_49.h5
data/modelnet/modelnet_graph_k15/chair/object_491.h5
data/modelnet/modelnet_graph_k15/chair/object_492.h5
data/modelnet/modelnet_graph_k15/chair/object_493.h5
data/modelnet/modelnet_graph_k15/chair/object_494.h5
data/modelnet/modelnet_graph_k15/chair/object_495.h5
data/modelnet/modelnet_graph_k15/chair/object_496.h5
data/modelnet/modelnet_graph_k15/chair/object_49

...

data/modelnet/modelnet_graph_k15/range_hood/object_127.h5
data/modelnet/modelnet_graph_k15/range_hood/object_128.h5
data/modelnet/modelnet_graph_k15/range_hood/object_132.h5
data/modelnet/modelnet_graph_k15/range_hood/object_133.h5
data/modelnet/modelnet_graph_k15/range_hood/object_135.h5
data/modelnet/modelnet_graph_k15/range_hood/object_136.h5
data/modelnet/modelnet_graph_k15/range_hood/object_137.h5
data/modelnet/modelnet_graph_k15/range_hood/object_139.h5
data/modelnet/modelnet_graph_k15/range_hood/object_14.h5
data/modelnet/modelnet_graph_k15/range_hood/object_140.h5
data/modelnet/modelnet_graph_k15/range_hood/object_141.h5
data/modelnet/modelnet_graph_k15/range_hood/object_143.h5
data/modelnet/modelnet_graph_k15/range_hood/object_144.h5
data/modelnet/modelnet_graph_k15/range_hood/object_145.h5
data/modelnet/modelnet_graph_k15/range_hood/object_147.h5
data/modelnet/modelnet_graph_k15/range_hood/object_148.h5
data/modelnet/modelnet_graph_k15/range_hood/object_149.h5
data/modelnet/m

data/modelnet/modelnet_graph_k15/range_hood/object_84.h5
data/modelnet/modelnet_graph_k15/range_hood/object_85.h5
data/modelnet/modelnet_graph_k15/range_hood/object_86.h5
data/modelnet/modelnet_graph_k15/range_hood/object_87.h5
data/modelnet/modelnet_graph_k15/range_hood/object_88.h5
data/modelnet/modelnet_graph_k15/range_hood/object_89.h5
data/modelnet/modelnet_graph_k15/range_hood/object_9.h5
data/modelnet/modelnet_graph_k15/range_hood/object_90.h5
data/modelnet/modelnet_graph_k15/range_hood/object_92.h5
data/modelnet/modelnet_graph_k15/range_hood/object_94.h5
data/modelnet/modelnet_graph_k15/range_hood/object_96.h5
data/modelnet/modelnet_graph_k15/range_hood/object_97.h5
data/modelnet/modelnet_graph_k15/range_hood/object_98.h5
data/modelnet/modelnet_graph_k15/range_hood/object_99.h5
lamp.h5
data/modelnet/modelnet_graph_k15/lamp/object_0.h5
data/modelnet/modelnet_graph_k15/lamp/object_1.h5
data/modelnet/modelnet_graph_k15/lamp/object_10.h5
data/modelnet/modelnet_graph_k15/lamp/object

data/modelnet/modelnet_graph_k15/toilet/object_125.h5
data/modelnet/modelnet_graph_k15/toilet/object_126.h5
data/modelnet/modelnet_graph_k15/toilet/object_127.h5
data/modelnet/modelnet_graph_k15/toilet/object_128.h5
data/modelnet/modelnet_graph_k15/toilet/object_129.h5
data/modelnet/modelnet_graph_k15/toilet/object_13.h5
data/modelnet/modelnet_graph_k15/toilet/object_130.h5
data/modelnet/modelnet_graph_k15/toilet/object_131.h5
data/modelnet/modelnet_graph_k15/toilet/object_133.h5
data/modelnet/modelnet_graph_k15/toilet/object_135.h5
data/modelnet/modelnet_graph_k15/toilet/object_136.h5
data/modelnet/modelnet_graph_k15/toilet/object_137.h5
data/modelnet/modelnet_graph_k15/toilet/object_138.h5
data/modelnet/modelnet_graph_k15/toilet/object_139.h5
data/modelnet/modelnet_graph_k15/toilet/object_14.h5
data/modelnet/modelnet_graph_k15/toilet/object_141.h5
data/modelnet/modelnet_graph_k15/toilet/object_142.h5
data/modelnet/modelnet_graph_k15/toilet/object_143.h5
data/modelnet/modelnet_graph_k

data/modelnet/modelnet_graph_k15/toilet/object_286.h5
data/modelnet/modelnet_graph_k15/toilet/object_287.h5
data/modelnet/modelnet_graph_k15/toilet/object_288.h5
data/modelnet/modelnet_graph_k15/toilet/object_289.h5
data/modelnet/modelnet_graph_k15/toilet/object_29.h5
data/modelnet/modelnet_graph_k15/toilet/object_290.h5
data/modelnet/modelnet_graph_k15/toilet/object_291.h5
data/modelnet/modelnet_graph_k15/toilet/object_292.h5
data/modelnet/modelnet_graph_k15/toilet/object_294.h5
data/modelnet/modelnet_graph_k15/toilet/object_295.h5
data/modelnet/modelnet_graph_k15/toilet/object_296.h5
data/modelnet/modelnet_graph_k15/toilet/object_297.h5
data/modelnet/modelnet_graph_k15/toilet/object_298.h5
data/modelnet/modelnet_graph_k15/toilet/object_299.h5
data/modelnet/modelnet_graph_k15/toilet/object_3.h5
data/modelnet/modelnet_graph_k15/toilet/object_30.h5
data/modelnet/modelnet_graph_k15/toilet/object_300.h5
data/modelnet/modelnet_graph_k15/toilet/object_301.h5
data/modelnet/modelnet_graph_k15

data/modelnet/modelnet_graph_k15/toilet/object_437.h5
data/modelnet/modelnet_graph_k15/toilet/object_438.h5
data/modelnet/modelnet_graph_k15/toilet/object_44.h5
data/modelnet/modelnet_graph_k15/toilet/object_440.h5
data/modelnet/modelnet_graph_k15/toilet/object_441.h5
data/modelnet/modelnet_graph_k15/toilet/object_443.h5
data/modelnet/modelnet_graph_k15/toilet/object_45.h5
data/modelnet/modelnet_graph_k15/toilet/object_46.h5
data/modelnet/modelnet_graph_k15/toilet/object_47.h5
data/modelnet/modelnet_graph_k15/toilet/object_48.h5
data/modelnet/modelnet_graph_k15/toilet/object_49.h5
data/modelnet/modelnet_graph_k15/toilet/object_50.h5
data/modelnet/modelnet_graph_k15/toilet/object_51.h5
data/modelnet/modelnet_graph_k15/toilet/object_52.h5
data/modelnet/modelnet_graph_k15/toilet/object_53.h5
data/modelnet/modelnet_graph_k15/toilet/object_54.h5
data/modelnet/modelnet_graph_k15/toilet/object_55.h5
data/modelnet/modelnet_graph_k15/toilet/object_57.h5
data/modelnet/modelnet_graph_k15/toilet/o